In [ ]:
!pip install nlpaug
!pip install nltk
!pip install sacremoses
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
import pandas as pd
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import MarianMTModel, MarianTokenizer
from datasets import Dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter, defaultdict
import math
import copy
import random
import operator
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
import time


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#Data Processing

In [ ]:
columns = ['id', 'identifier', 'category', 'country_code', 'text', 'multi_class_target']

try:
    data_df = pd.read_csv("/content/dontpatronizeme_pcl.tsv", sep='\t', header=None, names=columns, skiprows=3, index_col='id')
except pd.errors.ParserError as e:
    print("ParserError:", e)

data_df['target'] = data_df['multi_class_target'].apply(lambda x: 0 if x == 0 or x == 1 else 1)
data_df = data_df.dropna()
data_df["text_length"] = data_df["text"].apply(lambda x: len(x))

In [ ]:
data_df

,identifier,category,country_code,text,multi_class_target,target,text_length
id,,,,,,,
1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0,620
2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0,237
3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0,0,158
4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,162
5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0,0,273
...,...,...,...,...,...,...,...
10465,@@14297363,women,lk,Sri Lankan norms and culture inhibit women fro...,1,0,344
10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0,258
10467,@@20282330,in-need,ng,""" She has one huge platform , and information ...",3,1,282


In [ ]:
train_ids = pd.read_csv("/content/train_semeval_parids-labels.csv")
dev_ids = pd.read_csv("/content/dev_semeval_parids-labels.csv")

In [ ]:
train_df = data_df.loc[data_df.index.isin(train_ids['par_id'])]
dev_df = data_df.loc[data_df.index.isin(dev_ids['par_id'])]

In [ ]:
train_df

,identifier,category,country_code,text,multi_class_target,target,text_length
id,,,,,,,
1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0,620
2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0,237
3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0,0,158
4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,162
5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0,0,273
...,...,...,...,...,...,...,...
10424,@@4665292,women,jm,""" I do n't believe in abortion , I think it is...",3,1,133
10445,@@3923193,refugee,gb,More than 150 volunteers spent the night in ' ...,3,1,112
10454,@@22338535,vulnerable,ie,""" We are challenged , I suggest , to turn this...",4,1,240


In [ ]:
dev_df

,identifier,category,country_code,text,multi_class_target,target,text_length
id,,,,,,,
107,@@16900972,homeless,ke,"His present "" chambers "" may be quite humble ,...",3,1,394
149,@@1387882,disabled,us,Krueger recently harnessed that creativity to ...,2,1,296
151,@@19974860,poor-families,in,10:41am - Parents of children who died must ge...,3,1,138
154,@@20663936,disabled,ng,When some people feel causing problem for some...,4,1,496
157,@@21712008,poor-families,ca,We are alarmed to learn of your recently circu...,4,1,601
...,...,...,...,...,...,...,...
10463,@@4676355,refugee,pk,""" The Pakistani police came to our house and t...",0,0,166
10464,@@19612634,disabled,ie,When Marie O'Donoghue went looking for a speci...,0,0,199
10465,@@14297363,women,lk,Sri Lankan norms and culture inhibit women fro...,1,0,344


##Pre-Processing

In [ ]:
def clean_text(text):
    text = re.sub('n\'t', 'not', text)
    text = re.sub('\'s', '', text)
    text = re.sub(f'[{string.punctuation}]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    clean_text = ' '.join(tokens)
    return clean_text


In [ ]:
train_df["clean_text"] = train_df["text"].apply(
    lambda x: clean_text(x)
)

<ipython-input-14-e3aa0c659a8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["clean_text"] = train_df["text"].apply(


In [ ]:
train_df

,identifier,category,country_code,text,multi_class_target,target,text_length,clean_text
id,,,,,,,,
1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0,620,we live time absolut insan i pretti sure peopl...
2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0,237,in libya today countless number ghanaian niger...
3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0,0,158,white hous press secretari sean spicer said fo...
4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,162,council custom sign would display two space wo...
5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0,0,273,just like receiv migrant flee el salvador guat...
...,...,...,...,...,...,...,...,...
10424,@@4665292,women,jm,""" I do n't believe in abortion , I think it is...",3,1,133,i believ abort i think wick i would tell woman...
10445,@@3923193,refugee,gb,More than 150 volunteers spent the night in ' ...,3,1,112,more 150 volunt spent night surviv camp highli...
10454,@@22338535,vulnerable,ie,""" We are challenged , I suggest , to turn this...",4,1,240,we challeng i suggest turn time celebr sustain...


##Augmentation

In [ ]:
def back_translate(text, source_lang="en", target_lang="fr"):
    tokenizer_to = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}')
    model_to = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-{source_lang}-{target_lang}')

    translated_to = model_to.generate(**tokenizer_to([text], return_tensors="pt", padding=True))
    text_to = tokenizer_to.decode(translated_to[0], skip_special_tokens=True)

    tokenizer_back = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-{target_lang}-{source_lang}')
    model_back = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-{target_lang}-{source_lang}')

    translated_back = model_back.generate(**tokenizer_back([text_to], return_tensors="pt", padding=True))
    text_back = tokenizer_back.decode(translated_back[0], skip_special_tokens=True)

    return text_back

In [ ]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

aug = naf.Sometimes([
            naw.SynonymAug(aug_src='wordnet'),
            naw.ContextualWordEmbsAug(model_path='bert-base-cased', action="insert", device=device_type),
            naw.ContextualWordEmbsAug(model_path='bert-base-cased', action="substitute", device=device_type),
            nas.ContextualWordEmbsForSentenceAug(model_path='gpt2', device=device_type)
        ])

def augment_text(df, augmentor):
    all_data = [df]
    n = int(len(df[df["target"] == 0]) / len(df[df["target"] == 1])) if len(df[df["target"] == 1]) != 0 else 0
    n = n // 2
    print(f"Data augmentation: rebalancing {n} times...")
    for i in range(n):
        print(f"    Iteration {i+1}")
        start_time = time.time()
        df_new = df[df["target"] == 1].copy(deep=True)
        texts = df_new["clean_text"].tolist()
        augmented_text = [augmentor.augment(text)[0] for text in texts]
        df_new["clean_text"] = augmented_text
        all_data.append(df_new)
        print(f"Elapsed time is {int(time.time() - start_time)}s")
    return pd.concat(all_data, axis=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
train_df = augment_text(train_df, aug)

Data augmentation: rebalancing 4 times...
    Iteration 1


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Elapsed time is 254s
    Iteration 2
Elapsed time is 250s
    Iteration 3
Elapsed time is 251s
    Iteration 4
Elapsed time is 252s


In [ ]:
train_df

,identifier,category,country_code,text,multi_class_target,target,text_length,clean_text
id,,,,,,,,
3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0,0,158,white hous press secretari sean spicer said fo...
4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,162,council custom sign would display two space wo...
5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0,0,273,just like receiv migrant flee el salvador guat...
6,@@9382277,in-need,in,"To bring down high blood sugar levels , insuli...",0,0,361,to bring high blood sugar level insulin need t...
7,@@7562079,refugee,za,The European Union is making an historic mista...,0,0,301,the european union make histor mistak hast con...
...,...,...,...,...,...,...,...,...
10424,@@4665292,women,jm,""" I do n't believe in abortion , I think it is...",3,1,133,iodin believ abort but i would think it taper ...
10445,@@3923193,refugee,gb,More than 150 volunteers spent the night in ' ...,3,1,112,more and later others spent their night surviv...
10454,@@22338535,vulnerable,ie,""" We are challenged , I suggest , to turn this...",4,1,240,we challeng why i could suggest turn sort of m...


#Bag of Words Model

In [ ]:
# bag of words
# texts = data_df["text"].fillna("").tolist()
texts = train_df["text"].fillna("").tolist()

vectorizer = CountVectorizer()
x = vectorizer.fit_transform(texts)


y = []
for i in data_df["target"]:
  if i != 0 and i != 1:
    y.append(1)
  else: y.append(0)
y = train_df["target"]

train_df["target"][:30]

id
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
Name: target, dtype: int64

In [ ]:
y

id
1       0
2       0
3       0
4       0
5       0
       ..
7009    0
7010    0
7011    0
7012    0
7013    0
Name: target, Length: 6871, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# bag of words
# texts = data_df["text"].fillna("").tolist()
texts = train_df["text"].fillna("").tolist()

texts_array = np.array(texts)

vectorizer = CountVectorizer()
x = vectorizer.fit_transform(texts)

y = []
for i in data_df["target"]:
  if i != 0 and i != 1:
    y.append(1)
  else: y.append(0)
y = train_df["target"]

x_train, x_test, y_train, y_test, texts_train, texts_test = train_test_split(
    x.toarray(), y, texts_array, test_size=0.2, random_state=42)

# x_train, x_test, y_train, y_test = train_test_split(x.toarray(), y, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9083636363636364
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      1264
           1       0.14      0.03      0.05       111

    accuracy                           0.91      1375
   macro avg       0.53      0.51      0.50      1375
weighted avg       0.86      0.91      0.88      1375

Example of a misclassified text:
Brit bomber 's wife Shukee Begum on ' gangster mentality ' of ISIS women
Actual label: 0
Predicted label: 1


In [ ]:
# After fitting your model and making predictions
misclassified_indices = [i for i, (actual, predicted) in enumerate(zip(y_test, y_pred)) if actual != predicted]

if misclassified_indices:
    misclassified_index = misclassified_indices[5]
    misclassified_text = texts_test[misclassified_index]
    print("Example of a misclassified text:")
    print(misclassified_text)
    print(f"Actual label: {y_test.iloc[misclassified_index]}")
    print(f"Predicted label: {y_pred[misclassified_index]}")
else:
    print("No misclassifications found.")


Example of a misclassified text:
Brit bomber 's wife Shukee Begum on ' gangster mentality ' of ISIS women
Actual label: 0
Predicted label: 1


In [ ]:
Ptexts = dev_df["text"].fillna("").tolist()

vectorizer = CountVectorizer()
x = vectorizer.fit_transform(texts)
y_pred = clf.predict(x)
classification_report(dev_df["target"], y_pred)

ValueError: X has 12463 features, but MultinomialNB is expecting 26858 features as input.

#N-GRAM Model

In [ ]:
class NGramLM():
    def __init__(self, N):
        self.N = N
        self.vocab = set()
        self.data = []
        self.prob = {}
        self.counts = defaultdict(Counter)

    # For N = 1, the probability is stored in a dict   P = prob[next_word]
    # For N > 1, the probability is in a nested dict   P = prob[context][next_word]
    def train(self, vocab, data, smoothing_k=0):
        self.vocab = vocab
        self.data = data
        self.smoothing_k = smoothing_k

        if self.N == 1:
            self.counts = Counter(flatten(data))
            self.prob = self.get_prob(self.counts)
        else:
            self.vocab.add('<s>')
            counts = self.count_ngram()

            self.prob = {}
            for context, counter in counts.items():
                self.prob[context] = self.get_prob(counter)

    def count_ngram(self):
        counts = defaultdict(Counter)
        for sentence in self.data:
            sentence = (self.N - 1) * ['<s>'] + sentence
            for i in range(len(sentence)-self.N+1):
                context = sentence[i:i+self.N-1]
                context = " ".join(context)
                word = sentence[i+self.N-1]
                counts[context][word] += 1

        self.counts = counts
        return counts

    # normalize counts into probability(considering smoothing)
    def get_prob(self, counter):
        total = float(sum(counter.values()))
        k = self.smoothing_k

        prob = {}
        for word, count in counter.items():
            prob[word] = (count + k) / (total + len(self.vocab) * k)
        return prob

    def get_ngram_logprob(self, word, seq_len=1, context=""):
        if self.N == 1 and word in self.prob.keys():
            return math.log(self.prob[word]) / seq_len
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return math.log(self.prob[context][word]) / seq_len
        else:
            # assign a small probability to the unseen ngram
            # to avoid log of zero and to penalise unseen word or context
            return math.log(1/len(self.vocab)) / seq_len

    def get_ngram_prob(self, word, context=""):
        if self.N == 1 and word in self.prob.keys():
            return self.prob[word]
        elif self.N > 1 and not self._is_unseen_ngram(context, word):
            return self.prob[context][word]
        elif word in self.vocab and self.smoothing_k > 0:
            # probability assigned by smoothing
            return self.smoothing_k / (sum(self.counts[context].values()) + self.smoothing_k*len(self.vocab))
        else:
            # unseen word or context
            return 0

    # In this method, the perplexity is measured at the sentence-level, averaging over all sentences.
    # Actually, it is also possible to calculate perplexity by merging all sentences into a long one.
    def perplexity(self, test_data):
        log_ppl = 0
        if self.N == 1:
            for sentence in test_data:
                for word in sentence:
                    log_ppl += self.get_ngram_logprob(word=word, seq_len=len(sentence))
        else:
            for sentence in test_data:
                for i in range(len(sentence)-self.N+1):
                    context = sentence[i:i+self.N-1]
                    context = " ".join(context)
                    word = sentence[i+self.N-1]
                    log_ppl += self.get_ngram_logprob(context=context, word=word, seq_len=len(sentence))

        log_ppl /= len(test_data)
        ppl = math.exp(-log_ppl)
        return ppl

    def _is_unseen_ngram(self, context, word):
        if context not in self.prob.keys() or word not in self.prob[context].keys():
            return True
        else:
            return False

    # generate the most probable k words
    def generate_next(self, context, k):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)

        if ngram_context in self.prob.keys():
            candidates = self.prob[ngram_context]
            most_probable_words = sorted(candidates.items(), key=lambda kv: kv[1], reverse=True)
            for i in range(min(k, len(most_probable_words))):
                print(" ".join(context[self.N-1:])+" "+most_probable_words[i][0]+"\t P={}".format(most_probable_words[i][1]))
        else:
            print("Unseen context!")

    # generate the next n words with greedy search
    def generate_next_n(self, context, n):
        context = (self.N-1) * '<s> ' + context
        context = context.split()
        ngram_context_list = context[-self.N+1:]
        ngram_context = " ".join(ngram_context_list)

        for i in range(n):
            try:
                candidates = self.prob[ngram_context]
                most_likely_next = max(candidates.items(), key=operator.itemgetter(1))[0]
                context += [most_likely_next]
                ngram_context_list = ngram_context_list[1:] + [most_likely_next]
                ngram_context = " ".join(ngram_context_list)
            except:
                break
        print(" ".join(context[self.N-1:]))


In [ ]:
class InterpolateNGramLM(NGramLM):

    def __init__(self, N):
        super(InterpolateNGramLM, self).__init__(N)
        self.ngram_lms = []
        self.lambdas = []

    def train(self, vocab, data, smoothing_k=0, lambdas=[]):
        assert len(lambdas) == self.N
        assert sum(lambdas) - 1 < 1e-9
        self.vocab = vocab
        self.lambdas = lambdas

        for i in range(self.N, 0, -1):
            lm = NGramLM(i)
            print("Training {}-gram language model".format(i))
            lm.train(vocab, data, smoothing_k)
            self.ngram_lms.append(lm)

    def get_ngram_logprob(self, word, seq_len, context):
        prob = 1e-9
        for i, (coef, lm) in enumerate(zip(self.lambdas, self.ngram_lms)):
            context_words = context.split()
            cutted_context = " ".join(context_words[-self.N + i + 1:])
            prob += coef * lm.get_ngram_prob(context=cutted_context, word=word)
        return math.log(prob) / seq_len

In [ ]:
import numpy as np
flatten = lambda l: [item for sublist in l for item in sublist]
y = np.array(((data_df["target"] != 1) & (data_df["target"] != 0)).astype(int))

x_train, x_val, y_train, y_val = train_test_split(data_df["text"].fillna(""), y, test_size=0.2, random_state=42)
train_con = x_train[y_train==1]
train_norm = x_train[y_train!=1]
train_con_lists = []
train_norm_lists = []

for sentence in train_con:
  train_con_lists.append(sentence.split())
for sentence in train_norm:
  train_norm_lists.append(sentence.split())

con_corpus = flatten(train_con)
con_vocab = set(con_corpus)
norm_corpus = flatten(train_norm)
norm_vocab = set(norm_corpus)

In [ ]:
import numpy as np
flatten = lambda l: [item for sublist in l for item in sublist]
y = np.array(((data_df["target"] != 1) & (data_df["target"] != 0)).astype(int))

x_train, x_val, y_train, y_val = train_df["text"], dev_df["text"], train_df["target"],  dev_df["target"]
train_con = x_train[y_train==1]
train_norm = x_train[y_train!=1]
train_con_lists = []
train_norm_lists = []

for sentence in train_con:
  train_con_lists.append(sentence.split())
for sentence in train_norm:
  train_norm_lists.append(sentence.split())

con_corpus = flatten(train_con)
con_vocab = set(con_corpus)
norm_corpus = flatten(train_norm)
norm_vocab = set(norm_corpus)

In [ ]:
ilm_con = InterpolateNGramLM(3)
ilm_con.train(con_vocab, train_con_lists, lambdas=[0.5, 0.1,0.4])
ilm_norm = InterpolateNGramLM(3)
ilm_norm.train(norm_vocab, train_norm_lists, lambdas=[0.5,0.1,0.4])

Training 3-gram language model
Training 2-gram language model
Training 1-gram language model
Training 3-gram language model
Training 2-gram language model
Training 1-gram language model


In [ ]:
classif = np.zeros(y_val.shape)
dw1 = None
for i,line in enumerate(x_val):
  words = line.split()
  context = ""
  prob_con = 0
  prob_norm = 0
  for word in words:
    context +=  word + " "
    prob_con += ilm_con.get_ngram_logprob(context=context, word=word, seq_len=1)
    prob_norm += ilm_norm.get_ngram_logprob(context=context, word=word, seq_len=1)
  classif[i] = 1 if prob_con * 0.97 > prob_norm else 0

# to print a misclassified example
#   if np.array(y_val)[i] != classif[i]:
#     dw1 = line
#     print(classif[i])
#     print(np.array(y_val)[i])
#     break
# dw1

0.0
1


'His present " chambers " may be quite humble , but Shiyani has the tiny space very neatly organized and clean . Many people pass him by but do not manage to see him , because the space is partially hidden behind trees , which gives him a relative privacy . " There are many homeless sleeping around the station , " Captain Xoli Mbele , from the nearby Johannesburg Central Police station said .'

In [ ]:
print(np.sum(classif == y_val)/ len(y_val))
print(np.sum(0 == y_val)/ len(y_val) )
print(classification_report(y_val, classif))

0.22535211267605634
0.0
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.23      0.37       142

    accuracy                           0.23       142
   macro avg       0.50      0.11      0.18       142
weighted avg       1.00      0.23      0.37       142



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(np.sum(classif == y_test)/ len(y_test))
print(np.sum(0 == y_test)/ len(y_test) )
print(classification_report(y_test, classif))

ValueError: ('Lengths must match to compare', (1375,), (142,))

# Bert Base Cased Model

In [ ]:
x_train = list(train_df['clean_text'])
y_train = train_df['target'].values

x_dev = list(dev_df['text'])
y_dev = dev_df['target'].values

In [ ]:
type(dev_df['text'].values)

numpy.ndarray

In [ ]:
### BERT pretrained model

training_args = TrainingArguments(
    per_device_train_batch_size=8,  # Adjust batch size according to your GPU memory
    evaluation_strategy="epoch",
    logging_dir="./logs",
    output_dir="./results",
    num_train_epochs=6,  # Adjust the number of epochs as needed
)

# Load the pre-trained model and tokenizer
model_name = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

train_encodings = tokenizer(x_train, truncation="longest_first", padding=True) #padding="max_length"
val_encodings = tokenizer(x_dev, truncation="longest_first", padding=True)
train_encodings["labels"] = list(y_train)
val_encodings["labels"] = list(y_dev)

# Define the Trainer
bert_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= Dataset.from_dict(train_encodings),
    eval_dataset=Dataset.from_dict(val_encodings),
)

# Fine-tune the model
bert_trainer.train()

# Evaluate the fine-tuned model
results = bert_trainer.evaluate(eval_dataset=Dataset.from_dict(val_encodings))
print(results)

val_encodings.pop("labels")
y_pred = bert_trainer.predict(Dataset.from_dict(val_encodings))

classif = y_pred[0].argmax(axis=1)
print(classification_report(y_dev,classif))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.281000,6.596544


In [ ]:
# ### Toxic model pretrained classifier

# training_args = TrainingArguments(
#     per_device_train_batch_size=8,  # Adjust batch size according to your GPU memory
#     evaluation_strategy="epoch",
#     logging_dir="./logs",
#     output_dir="./results",
#     num_train_epochs=1,  # Adjust the number of epochs as needed
# )

# model_name = "unitary/toxic-bert"
# tokenizer = AutoTokenizer.from_pretrained(model_name, num_labels=1)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)

# train_encodings = tokenizer(x, truncation="longest_first", padding=True) #padding="max_length"
# val_encodings = tokenizer(x_v, truncation="longest_first", padding=True)
# train_encodings["labels"] = list(torch.tensor(y_train, dtype=torch.float).unsqueeze(-1))
# val_encodings["labels"] = list(torch.tensor(y_val, dtype=torch.float).unsqueeze(-1))

# # Define the Trainer
# toxic_trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=Dataset.from_dict(train_encodings),
#     eval_dataset=Dataset.from_dict(val_encodings),
# )

# # Fine-tune the model
# toxic_trainer.train()

# # Evaluate the fine-tuned model
# results = toxic_trainer.evaluate(eval_dataset=Dataset.from_dict(val_encodings))
# print(results)

# # Predict
# val_encodings.pop("labels")  # Remove labels for prediction
# y_pred = toxic_trainer.predict(Dataset.from_dict(val_encodings))
# # classif = y_pred.predictions.argmax(axis=1)
# classif = y_pred[0].argmax(axis=1)

# # Assuming you have `classification_report` from `sklearn.metrics`
# from sklearn.metrics import classification_report
# print(classification_report(y_val, classif))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([6, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.323200,0.347332


{'eval_loss': 0.34733203053474426, 'eval_runtime': 47.5985, 'eval_samples_per_second': 43.993, 'eval_steps_per_second': 5.504, 'epoch': 1.0}
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1883
           1       0.00      0.00      0.00       211

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.47      2094
weighted avg       0.81      0.90      0.85      2094



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# ### Toxic model pretrained classifier

# training_args = TrainingArguments(
#     per_device_train_batch_size=8,  # Adjust batch size according to your GPU memory
#     evaluation_strategy="epoch",
#     logging_dir="./logs",
#     output_dir="./results",
#     num_train_epochs=4,  # Adjust the number of epochs as needed
# )

# model_name = "microsoft/deberta-large"
# tokenizer = AutoTokenizer.from_pretrained(model_name, num_labels=1)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)

# train_encodings = tokenizer(x, truncation="longest_first", padding=True) #padding="max_length"
# val_encodings = tokenizer(x_v, truncation="longest_first", padding=True)
# train_encodings["labels"] = list(torch.tensor(y_train, dtype=torch.float).unsqueeze(-1))
# val_encodings["labels"] = list(torch.tensor(y_val, dtype=torch.float).unsqueeze(-1))

# # Define the Trainer
# toxic_trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=Dataset.from_dict(train_encodings),
#     eval_dataset=Dataset.from_dict(val_encodings),
# )

# # Fine-tune the model
# toxic_trainer.train()

# # Evaluate the fine-tuned model
# results = toxic_trainer.evaluate(eval_dataset=Dataset.from_dict(val_encodings))
# print(results)

# # Predict
# val_encodings.pop("labels")  # Remove labels for prediction
# y_pred = toxic_trainer.predict(Dataset.from_dict(val_encodings))
# # classif = y_pred.predictions.argmax(axis=1)
# classif = y_pred[0].argmax(axis=1)

# # Assuming you have `classification_report` from `sklearn.metrics`
# from sklearn.metrics import classification_report
# print(classification_report(y_val, classif))


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.086400,0.090897
2,0.090000,0.090655
3,0.088600,0.091483
4,0.080200,0.090837


{'eval_loss': 0.09083667397499084, 'eval_runtime': 45.3769, 'eval_samples_per_second': 46.147, 'eval_steps_per_second': 5.774, 'epoch': 4.0}
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1883
           1       0.00      0.00      0.00       211

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.47      2094
weighted avg       0.81      0.90      0.85      2094



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
torch.cuda.empty_cache()

# Custom Model

In [ ]:
### BERT custom pretrained model
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn

torch.cuda.empty_cache()

# Adjust training arguments as needed
training_args = TrainingArguments(
    per_device_train_batch_size=4,  # May need to lower batch size due to larger model size
    evaluation_strategy="epoch",
    logging_dir="./logs",
    output_dir="./results",
    num_train_epochs=3,  # Adjust the number of epochs as needed
)

# Load the pre-trained model and tokenizer
model_name = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class CustomBERT(nn.Module):
    def __init__(self, model_name, num_labels):
        super(CustomBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        # Example of adding custom layers
        self.dropout = nn.Dropout(0.2)
        # Adjust classifier to use the correct hidden size for BERT large
        self.classifier = nn.Linear(1024, num_labels)  # BERT large has a hidden size of 1024

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # For BERT, use `pooler_output` for the [CLS] token representation if available
        # outputs.pooler_output directly gives the [CLS] token representation
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        print(f"Logits Shape: {logits.shape}")
        return logits

# Assuming x, x_v, y_train, y_val are defined elsewhere
# Continue with your dataset preparation and training as before


model = CustomBERT(model_name=model_name, num_labels=1)

train_encodings = tokenizer(x, truncation="longest_first", padding=True) #padding="max_length"
val_encodings = tokenizer(x_v, truncation="longest_first", padding=True)
train_encodings["labels"] = list(torch.tensor(y_train, dtype=torch.float).unsqueeze(-1))
val_encodings["labels"] = list(torch.tensor(y_val, dtype=torch.float).unsqueeze(-1))
print(f"First train encoding input IDs shape: {torch.tensor(train_encodings['input_ids'][0]).shape}")

# Example of inspecting the shape of labels for the entire training set (assuming y_train is a list)
print(f"Train labels shape: {torch.tensor(y_train).unsqueeze(-1).shape}")


# Define the Trainer
bert_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= Dataset.from_dict(train_encodings),
    eval_dataset=Dataset.from_dict(val_encodings),
)

# Fine-tune the model
bert_trainer.train()

# Evaluate the fine-tuned model
results = bert_trainer.evaluate(eval_dataset=Dataset.from_dict(val_encodings))
print(results)

val_encodings.pop("labels")
y_pred = bert_trainer.predict(Dataset.from_dict(val_encodings))

classif = y_pred[0].argmax(axis=1)
print(classification_report(y_val,classif))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

First train encoding input IDs shape: torch.Size([512])
Train labels shape: torch.Size([8375, 1])
Logits Shape: torch.Size([4, 1])


RuntimeError: output with shape [] doesn't match the broadcast shape [1]